## Imports

In [1]:
from openai import OpenAI
import json
import os
import secret_keys
%load_ext autoreload
%autoreload 2

In [2]:
def positive_function(phrase, reason=None):
    """
    Prints a message indicating that the given phrase is positive, 
    along with an optional explanation.

    Parameters
    ----------
    phrase : str
        The phrase that is considered positive.
    reason : str, optional
        An explanation or justification for why the phrase is positive.
        If not provided, a default message will be shown.

    Returns
    -------
    None
        This function only prints output and does not return a value.
    """
    print(f"The following phrase ({phrase}), was positive")
    if reason is not None:
        print(f"The reason: {reason}")
    else:
        print("No Reasoning was provided")


def negative_function(phrase, reason=None):
    """
    Prints a message indicating that the given phrase is negative, 
    along with an optional explanation.

    Parameters
    ----------
    phrase : str
        The phrase that is considered negative.
    reason : str, optional
        An explanation or justification for why the phrase is negative.
        If not provided, a default message will be shown.

    Returns
    -------
    None
        This function only prints output and does not return a value.
    """
    print(f"The following phrase ({phrase}), was negative")
    if reason is not None:
        print(f"The reason: {reason}")
    else:
        print("No Reasoning was provided")

In [3]:
FUNCTION_SCHEMA = [
    {
        "name": "positive_function",
        "description": "Prints a message indicating that the given phrase is positive,along with an optional explanation.",
        "parameters": {
            "type": "object",
            "properties": {
                "phrase": {"type": "string", "description": "The phrase that is considered positive."},
                "reason": {"type": "string", "description": "An explanation or justification for why the phrase is positive.If not provided, a default message will be shown. (Optional)"
        },
        },
            "required": ["phrase"]
        }
    },
    {
        "name": "negative_function",
        "description": "Prints a message indicating that the given phrase is positive, along with an optional explanation."
    ,
        "parameters": {
            "type": "object",
            "properties": {
                "phrase": {"type": "string", "description": "The phrase that is considered negative."},
                "reason": {"type": "string", "description": "An explanation or justification for why the phrase is negative.If not provided, a default message will be shown. (Optional)"
        },
            },
            "required": ["phrase"]
        }
    }
]

In [4]:
def call_openai(prompt,client):
    """
    Calls the OpenAI API with the given prompt and function schema.
    
    Args:
        prompt (str): The user's prompt
    
    Returns:
        dict: The API response
    """
    response = client.chat.completions.create(
        model="gpt-oss-120b:free",  # You can change the model as needed
        messages=[{"role": "user", "content": prompt}],
        tools=[{"type": "function", "function": func} for func in FUNCTION_SCHEMA],
        tool_choice="auto"
    )
    return response

In [5]:
def execute_function(tool_call):
    """
    Executes the function specified in the API response.
    
    Args:
        tool_call (dict): The tool call details from the API
    
    Returns:
        str: The result of the function execution
    """
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    if function_name == "positive_function":
        return positive_function(
            phrase = arguments["phrase"],
            reason = arguments["reason"]
        )
    elif function_name == "negative_function":
        return negative_function(
            phrase = arguments["phrase"],
            reason = arguments["reason"]
        )
    else:
        return "Error: Unknown function"

In [8]:
from secret_keys import OPENROUTER_API_KEY ,OPENROUTER_API_BASE
def main():
    # Example prompts
    instructions = "Given a prompt, use the appropriate function and pass the appropriate parameters to it the prompt: "
    prompt1 = "Chocolate tastes good, I like it."
    prompt2 = "Grapes taset quite bad frankly, even tho some people like them"
    prompts = [
        instructions+prompt1,
        instructions+prompt2,
    ]

    client = OpenAI(
        api_key=OPENROUTER_API_KEY,  # Replace with your actual OpenRouter API key
        base_url=OPENROUTER_API_BASE
    )
    for prompt in prompts:
        print("------------------------------------------------------------")
        print(f"\nPrompt: {prompt}")
        try:
            response = call_openai(prompt=prompt,client=client)
            message = response.choices[0].message
            
            if message.tool_calls:
                result = execute_function(message.tool_calls[0])
                print(f"Function execution result: {result}")
            else:
                print(f"Response: {message.content}")
        except Exception as e:
            print(f"Error: {str(e)}")
        print("------------------------------------------------------------")

In [9]:
main()

------------------------------------------------------------

Prompt: Given a prompt, use the appropriate function and pass the appropriate parameters to it the prompt: Chocolate tastes good, I like it.
The following phrase (Chocolate tastes good, I like it.), was positive
The reason: The statement expresses enjoyment and a positive opinion about chocolate.
Function execution result: None
------------------------------------------------------------
------------------------------------------------------------

Prompt: Given a prompt, use the appropriate function and pass the appropriate parameters to it the prompt: Grapes taset quite bad frankly, even tho some people like them
The following phrase (Grapes taste quite bad frankly, even though some people like them), was negative
The reason: The statement expresses a negative opinion about the taste of grapes.
Function execution result: None
------------------------------------------------------------
